In [22]:
import openai
import json
import re
from pydantic import BaseModel
from typing import Optional, List

# Updated client for new OpenAI library
client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama"  # dummy key
)

# Define your schema
class Pet(BaseModel):
    name: str
    animal: str
    age: int
    color: Optional[str]
    favorite_toy: Optional[str]

class PetList(BaseModel):
    pets: List[Pet]

# Prompt
prompt = """
I have two pets.
A cat named Luna who is 5 years old and loves playing with yarn. She has grey fur.
I also have a 2 year old black cat named Loki who loves tennis balls.

Please return their data as JSON in this format:

{
  "pets": [
    {
      "name": "Luna",
      "animal": "cat",
      "age": 5,
      "color": "grey",
      "favorite_toy": "yarn"
    },
    {
      "name": "Loki",
      "animal": "cat",
      "age": 2,
      "color": "black",
      "favorite_toy": "tennis balls"
    }
  ]
}
Only return valid JSON. No explanation. No markdown.
"""

try:
    response = client.chat.completions.create(
        model="llama3.2:3b-instruct-fp16",  # or whatever you have installed
        temperature=0.2,
        messages=[{"role": "user", "content": prompt}]
    )

    output = response.choices[0].message.content
    print("Raw output from model:\n", output)

    # Try to extract JSON from output
    match = re.search(r'\{[\s\S]*\}', output)
    if match:
        cleaned_output = match.group(0)
    else:
        raise ValueError("No valid JSON object found in model output.")

    # Validate with Pydantic
    data = json.loads(cleaned_output)
    pets = PetList(**data)
    print("\nParsed result:\n", pets)

except Exception as e:
    print("Error:", e)


Raw output from model:
 {"pets":[{"name":"Luna","animal":"cat","age":5,"color":"grey","favorite_toy":"yarn"},{"name":"Loki","animal":"cat","age":2,"color":"black","favorite_toy":"tennis balls"}]}

Parsed result:
 pets=[Pet(name='Luna', animal='cat', age=5, color='grey', favorite_toy='yarn'), Pet(name='Loki', animal='cat', age=2, color='black', favorite_toy='tennis balls')]


We can also directly pass the requested response format to the model.

# Simple classification example

In [23]:
from ollama import chat
from pydantic import BaseModel
from typing import List, Literal, Optional

class Object(BaseModel):
    name: str
    confidence: float
    attributes: str

class ImageDescription(BaseModel):
    summary: str
    objects: List[Object]
    scene: str
    colors: List[str]
    time_of_day: Literal['Morning', 'Afternoon', 'Evening', 'Night']
    setting: Literal['Indoor', 'Outdoor', 'Unknown']
    text_content: Optional[str] = None

path = 'files/beach.jpg'

response = chat(
    model='llama3.2-vision',
    format=ImageDescription.model_json_schema(),
    messages=[
        {
            'role': 'user',
            'content': 'Analyze this image and describe what you see, including any objects, the scene, colors and any text you can detect.',
            'images': [path],
        },
    ],
    options={'temperature': 0.7},
)

#  Fixed: Access the dict correctly
image_description = ImageDescription.model_validate_json(response['message']['content'])

print(json.dumps(image_description.model_dump(), indent=2))

{
  "summary": "The image features a serene beach scene with a calm ocean and a clear blue sky, evoking a sense of relaxation and tranquility.",
  "objects": [
    {
      "name": "Ocean",
      "confidence": 0.9,
      "attributes": "calm, blue"
    },
    {
      "name": "Sky",
      "confidence": 0.9,
      "attributes": "clear, blue"
    },
    {
      "name": "Beach",
      "confidence": 0.8,
      "attributes": "serene, sandy"
    }
  ],
  "scene": "beach",
  "colors": [
    "blue",
    "brown",
    "green"
  ],
  "time_of_day": "Morning",
  "setting": "Outdoor",
  "text_content": null
}


# adding object counters

In [ ]:
import json
from pydantic import BaseModel
from typing import List, Literal, Optional
from ollama import chat

class Object(BaseModel):
    name: str
    confidence: float
    attributes: str

class GenderCount(BaseModel):
    gender: Literal['Male', 'Female', 'Non-binary', 'Unknown']
    count: int
    confidence: Optional[float] = None

class PersonSummary(BaseModel):
    total_people: int
    gender_breakdown: List[GenderCount]
    notes: Optional[str] = None

class ImageDescription(BaseModel):
    summary: str
    setting: Literal['Indoor', 'Outdoor', 'Unknown']
    text_content: Optional[str] = None
    person_summary: Optional[PersonSummary] = None

# Path to your image
path = 'files/thiemo.JPG'

# Request output in JSON format
response = chat(
    model='llama3.2-vision',
    format='json',  # ✅ valid usage
    messages=[
        {
            'role': 'user',
            'content': 'Analyze this image and describe what you see in the following JSON format:\n' + 
                       json.dumps(ImageDescription.model_json_schema(), indent=2) + 
                       '\nOnly return valid JSON, no explanation.',
            'images': [path],
        },
    ],
    options={'temperature': 0},
)

# Parse the result into the Pydantic model
image_description = ImageDescription.model_validate_json(response['message']['content'])

# Print formatted output
print(json.dumps(image_description.model_dump(), indent=2))
